**Тестирование на NER датасете с использованием rubert**

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("unimelb-nlp/wikiann", 'ru')

In [2]:
print(raw_datasets['train'][0]["tokens"])
print(raw_datasets['train'][0]["ner_tags"])
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature # уже список тегов

['Илизаров', ',', 'Гавриил', 'Абрамович']
[1, 2, 2, 2]


Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)

In [3]:
# Все лейблы, нужны будут для evaluate
all_labels = ner_feature.feature.names

# Подготовка датасета

In [4]:
from transformers import AutoTokenizer

# Нельзя просто rubert-base-cased, нужно, чтобы
# hf.co/model_name резолвилось.
model_checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
SPECIAL_TOKEN_IDX = -100 # игнорируется функцией потерь

# Суть в том, что после токенизации берта слова разбились на части
# типа ["Гав", "##рил", "##ов"], а у них должна быть одинаковая метка
# поэтому нужен алайнмент
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    cur_wid = None
    for wid in word_ids:
        # Начало нового
        if wid != cur_wid:
            cur_wid = wid
            label = -100 if (wid is None) else labels[wid]
            
            new_labels.append(label)
        elif wid is None:
            new_labels.append(-100)
        
        # продолжение текущего
        else:
            label = labels[cur_wid]
            if label % 2 == 1:
                label += 1 # случай смены с I-, на B-
                            
            new_labels.append(label)
            
    return new_labels

In [6]:
# Батчевая версия для токенизатора
def tokenize_and_align_labels(examples):
    # Пока нельзя return_tensors, так как пэддинг будет неправильный
    # Это настроим в коллаторе
    tokenized_inputs = tokenizer(examples['tokens'], truncation=True, is_split_into_words=True)
    
    new_labels = []
    
    for i, cur_labels in enumerate(examples['ner_tags']):
        cur_word_ids = tokenized_inputs.word_ids(i)
        aligned_labels = align_labels_with_tokens(cur_labels, cur_word_ids)
        new_labels.append(aligned_labels)
    tokenized_inputs['labels'] = new_labels
    
    return tokenized_inputs

tokenize_and_align_labels(raw_datasets['train'][0, 1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 35377, 31332, 1388, 128, 56031, 41439, 102], [101, 118, 118, 118, 21919, 30310, 5679, 118, 118, 118, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 1, 2, 2, 2, 2, 2, -100], [-100, 0, 0, 0, 5, 6, 6, 0, 0, 0, -100]]}

In [7]:
import torch
from transformers import AutoModelForTokenClassification

# Инициализируем модель, указываем,
# сколько выходов будет у новой головы
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(all_labels)) 

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

model.config.num_labels

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


7

In [8]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    # выкинем ner_tags, spans, lan и тд
    remove_columns=raw_datasets["train"].column_names
)

# Инференс модели + тестирование eval

In [9]:
# Для начала нужно завести DataCollator
# есть как раз готовый, который -100 будет заполнять лейблы

from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

val_dataloader = DataLoader(
    tokenized_datasets['validation'], batch_size=256, shuffle=False,
    collate_fn=data_collator
)

In [10]:
from src.evaluation.evaluator import TokenClassificationEvaluator

model.eval()
evaluator = TokenClassificationEvaluator(
    all_labels, label_ids_to_ignore=[SPECIAL_TOKEN_IDX]
)
for batch in val_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    # Позволяет накапливать по батчам, а потом считать сразу за все
    evaluator.add_batch(model_outputs=outputs, references=batch['labels'])
    
f1_score = evaluator.compute()['overall_f1']

In [11]:
f1_score

0.01415979889903586

**Скор получился низким, так как у модели голова инициализирована рандомно**